# Example implementation of the PAMpy framework

In this script we will walk through an example of how you can build, use and change the *Escherichia Coli* (*E.coli*) proteome allocation model (PAM) using the PAMpy framework. The principles behind the PAM are described by Alter et al (2021)[1]

[1]. Alter TB, Blank LM, Ebert BE. Proteome Regulation Patterns Determine Escherichia coli Wild-Type and Mutant Phenotypes. Elias JE, ed. mSystems. 2021;6(2):e00625-20. doi:10.1128/mSystems.00625-20


In [1]:
#importing the packages and scripts
import os
from cobra.io import read_sbml_model, load_matlab_model
import sys
import pandas as pd

#load PAMpy modules
from PAModelpy.EnzymeSectors import ActiveEnzymeSector, TransEnzymeSector, UnusedEnzymeSector
from PAModelpy.PAModel import PAModel
from PAModelpy.PAMValidator import PAMValidator
from PAModelpy.configuration import Config

Loading PAModelpy modules version 0.0.3.3


In [3]:
#Setting the relative paths
BASE_DIR = os.path.split(os.getcwd())[0]
MODEL_DIR = os.path.join(BASE_DIR, 'Models')
DATA_DIR = os.path.join(BASE_DIR, 'Data')
pam_info_file = os.path.join(DATA_DIR, 'proteinAllocationModel_iML1515_EnzymaticData_py.xls')
VALID_DATA_DIR = os.path.join(DATA_DIR, 'Ecoli_phenotypes')
valid_data_file = os.path.join(VALID_DATA_DIR, 'Ecoli_phenotypes_py_rev.xls')
valid_data_file_glyc = os.path.join(VALID_DATA_DIR, 'Ecoli_phenotypes_py_glyc.xls')

RESULT_DIR = os.path.join(BASE_DIR, 'Results')

#some other constants
TOTAL_PROTEIN_CONCENTRATION = 0.258  # [g_prot/g_cdw]

## 1. Building a PAModel from the existing IML1515 *E.coli* model

The PAM will be build using information about which enzymes catalyzes a specific reaction, the turnover rate of the catalysis and the molar mass of the enzyme (active enzyme information), information about the proportion of translational enzymes and information about what proportion of enzymes is not used (unused enzymes). The sum of all the enzyme sectors is limited to a user defined total protein fraction of biomass. For prokaryotes, the protein fraction of biomass is assumed to be constant.

In [4]:
#load active enzyme sector information
active_enzyme_info = pd.read_excel(pam_info_file, sheet_name='ActiveEnzymes')

# replace NaN values with unique identifiers
#select the NaN values 
nan_values = active_enzyme_info['EC_nmbr'].isnull()
#make a list with unique ids
nan_ids = [f'E{i}' for i in range(nan_values.sum())]
#replace nan values by unique id
active_enzyme_info.loc[nan_values, 'EC_nmbr'] = nan_ids

#check if it worked:
active_enzyme_info[nan_values]

,rxnID,rxnName,rxnEquat,EC_nmbr,molMass,kcat
557,GLUtex_f,L-glutamate transport via diffusion (extracell...,glu__L_e -> glu__L_p,E0,39959.4825,100000000.0
558,GLYtex_f,Glycine transport via diffusion (extracellular...,gly_e -> gly_p,E1,39959.4825,100000000.0
559,GLYALDtex_f,Glyceraldehyde transport via diffusion (extrac...,glyald_e -> glyald_p,E2,39959.4825,100000000.0
560,GLYBtex_f,Glycine betaine transport via diffusion (extra...,glyb_e -> glyb_p,E3,39959.4825,100000000.0
561,GLYCtex_f,Glycerol transport via diffusion (extracellula...,glyc_e -> glyc_p,E4,39959.4825,100000000.0
...,...,...,...,...,...,...
2837,METGLCURtex_b,1-O-methyl-Beta-D-glucuronate via diffusion (e...,metglcur_p -> metglcur_e,E953,39959.4825,100000000.0
2838,AI2tex_b,"Ai2 transport, outer membrane",mththf_p -> mththf_e,E954,39959.4825,22.0
2840,ASPtpp_b,L-aspartate uptake via facillitated diffusion,asp__L_c -> asp__L_p,E955,59427.5800,100000000.0
2841,FUMt1pp_b,Fumarate transport via diffusion in (periplasm),fum_c -> fum_p,E956,59427.5800,100000000.0


In [5]:
# parse the enzyme information (kcat values, identifiers and molmasses)
kcats_dict = active_enzyme_info.set_index(keys='rxnID').loc[:, 'kcat'].to_dict()
ec_dict = active_enzyme_info.set_index(keys='rxnID').loc[:, 'EC_nmbr'].to_dict()
molmass_dict = mol_mass=active_enzyme_info.set_index(keys='rxnID').loc[:,'molMass'].to_dict()


kcats = {}
# save fwd and bckw kcats separately in the form of: {rxn_id: {'f': kcat_f, 'b': kcat_b}}
for rxn, kcat in kcats_dict.items():
    #reversible reaction
    if rxn[-2:] == '_f' or rxn[-2:] == '_b':
        direction = rxn[-1]
        #check if the reaction already exists in the kcat dictionary
        try: 
            kcats[rxn[:-2]][direction] = kcat
        except:
            kcats[rxn[:-2]] = {direction: kcat}
    #irreversible reaction
    else:
        kcats[rxn] = {'f': kcat}

rxn2ec = {}
#parse the enzyme identifiers for the reactions
for rxn, ec in ec_dict.items():
    if rxn[-2:] == '_f' or rxn[-2:] == '_b':
        rxn = rxn[:-2]
    for enz in str(ec).split(','):
        rxn2ec[rxn] = enz.strip()
        
molmass = {}
#parse the enzyme molmasses for the reactions
for rxn, mw in molmass_dict.items():
    if rxn[-2:] == '_f' or rxn[-2:] == '_b':
        rxn = rxn[:-2]
    molmass[rxn] = mw
    
rxn2protein = {}
for rxn, ec in rxn2ec.items():
    ec_dict = {**kcats[rxn], **{'molmass': molmass[rxn]}}
    #add enzyme to enzymes related to reaction if these are already stored
    if rxn in rxn2protein.keys():
        rxn2protein[rxn] = {**rxn2protein[rxn], **{ec:ec_dict}}
    #if not create new reaction entry
    else:
        rxn2protein[rxn] = {ec:ec_dict}

active_enzyme_sector = ActiveEnzymeSector(rxn2protein=rxn2protein)

In [6]:
#load the translational enzyme information
translational_info = pd.read_excel(pam_info_file, sheet_name='Translational')
id_list = [translational_info[translational_info.Parameter == 'id_list'].loc[0,'Value']]
translation_enzyme_sector = TransEnzymeSector(id_list=id_list,
                                                tps_0=[translational_info[translational_info.Parameter == 'tps_0'].loc[1,'Value']],
                                                tps_mu=[translational_info[translational_info.Parameter == 'tps_mu'].loc[2,'Value']],
                                                mol_mass=[translational_info[translational_info.Parameter == 'mol_mass'].loc[3,'Value']])

In [7]:
#load the unused enzyme information
unused_protein_info = pd.read_excel(pam_info_file, sheet_name='ExcessEnzymes')

ups_0 = unused_protein_info[unused_protein_info.Parameter == 'ups_0'].loc[2,'Value']
smax = unused_protein_info[unused_protein_info.Parameter == 's_max_uptake'].loc[1,'Value']
id_list =[unused_protein_info[translational_info.Parameter == 'id_list'].loc[0,'Value']]


unused_protein_sector = UnusedEnzymeSector(id_list=id_list,
                                            ups_mu=[ups_0/smax],
                                            ups_0=[ups_0],
                                            mol_mass=[unused_protein_info[unused_protein_info.Parameter == 'mol_mass'].loc[3,'Value']])

In [8]:
#load the genome-scale information
model = read_sbml_model(os.path.join(MODEL_DIR, 'iML1515.xml'))

#load the PAM with the genome-scale information and the information about the enzyme sectors
pamodel = PAModel(id_or_model=model, 
                  p_tot=TOTAL_PROTEIN_CONCENTRATION,
                   active_sector=active_enzyme_sector, 
                  translational_sector=translation_enzyme_sector,
                  unused_sector=unused_protein_sector,
                        sensitivity =True
                 )

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-07
Read LP format model from file /tmp/tmpl8hgnuhf.lp
Reading time = 0.01 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Setting up the proteome allocation model iML1515

Add total condition-dependent protein constraint
	Total protein concentration: 0.258 g/gDW

Add active protein sector



/home/samiralvdb/.local/lib/python3.10/site-packages/PAModelpy/PAModel.py:222: UserWarning: Molar mass for E332 is invalid: 0.0
  warnings.warn(f'Molar mass for {enz.id} is invalid: {molmass}')


Add the following protein sector: TranslationalProteinSector

Add the following protein sector: 

Done with setting up the proteome allocation model iML1515



## 2. Testing and validating the PAM
To see if the PAM is working we can run some dummy simulations. Also, the PAMValidator module has functions which allow for easy visualization of the model predictions vs measured experimental data.

In [9]:
#first test the model
pamodel.test()

Protein allocation model test case was successful.

Optimal objective function value:  0.7479665512499333 

Predicted glucose uptake rate:  -10.0


In [10]:
#test the model with a different glucose uptake
pamodel.test(5)

Protein allocation model test case was successful.

Optimal objective function value:  0.40412477292088256 

Predicted glucose uptake rate:  -5.0


In [12]:
from PAModelpy.PAMValidator import PAMValidator

validator = PAMValidator(pamodel, valid_data_file)
#model flux rates of biomass formation, acetate, CO2 and O2 vs glucose uptake rate for a range of growth rates
validator.validate_range()

Running simulations to validate the model...



/home/samiralvdb/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations

Done running simulations



,Y_biomass,Y_acetate,Y_co2,BIOMASS_Ec_iML1515_core_75p37M,EX_glc__D_e,EX_ac_e,EX_co2_e,EX_o2_e,PGI,G6PDH2r,EDA,CS,ICL,PPC,ME1,ME2
0,-0.206710,-0.000000,-24.819544,0.018620,-0.5,0.000000,2.235693,-2.195954,0.500000,0.000000,0.0,0.738792,0.055111,0.000000,0.0,0.0
1,-0.346899,-0.000000,-19.065120,0.062496,-1.0,0.000000,3.434693,-3.301312,1.000000,0.000000,0.0,1.123284,0.184974,0.000000,0.0,0.0
2,-0.393588,-0.000000,-17.148651,0.106361,-1.5,0.000000,4.634146,-4.407146,1.500000,0.000000,0.0,1.507932,0.314804,0.000000,0.0,0.0
3,-0.416891,-0.000000,-16.192092,0.150211,-2.0,0.000000,5.834201,-5.513615,2.000000,0.000000,0.0,1.892785,0.444591,0.000000,0.0,0.0
4,-0.429852,-0.000000,-15.660067,0.193601,-2.5,0.000000,7.053133,-6.639941,2.385339,0.114661,0.0,2.332982,0.649562,0.000000,0.0,0.0
5,-0.437965,-0.000000,-15.327065,0.236706,-3.0,0.000000,8.283783,-7.778595,2.859810,0.140190,0.0,2.851925,0.906933,0.000000,0.0,0.0
6,-0.443571,-0.000000,-15.096971,0.279691,-3.5,0.000000,9.519328,-8.922399,3.092150,0.407850,0.0,3.294942,1.167070,0.000000,0.0,0.0
7,-0.447675,-0.000000,-14.928494,0.322605,-4.0,0.000000,10.757824,-10.069306,3.228942,0.771058,0.0,3.710521,1.430399,0.000000,0.0,0.0
8,-0.450807,-0.000000,-14.799933,0.365470,-4.5,0.000000,11.998327,-11.218326,3.376947,1.123053,0.0,4.126325,1.689388,0.000000,0.0,0.0
9,-0.446627,-0.000000,-14.971491,0.402313,-5.0,0.000000,13.486010,-12.627377,4.301473,0.698527,0.0,4.814257,0.197746,1.661949,0.0,0.0


Plotting the results against reference values. 



NameError: name 'configuration' is not defined

In [11]:
#you can also add custom reactions or variables which you would like to store and save them to a csv file.
#for example the translationalProteinSector variable
validator.add_rxn_to_save("FUM") #should remain constant
validator.validate_range(show=False, save=True)

#or plot the custom reactions directly
validator.validate_range(pfba = False, show=True, save=False)

/home/samiralvdb/Documents/3_Projects/2_PAMpy/2_PAMpy_Gitlab/Examples/../Package/PAMpy/PAMValidator.py:105: FutureWarning:

This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.

/home/samiralvdb/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning:

Solver status is 'infeasible'.



Running simulations to validate the model...



,Y_biomass,Y_acetate,Y_co2,BIOMASS_Ec_iML1515_core_75p37M,EX_glc__D_e,EX_ac_e,EX_co2_e,EX_o2_e,PGI,G6PDH2r,EDA,CS,ICL,PPC,ME1,ME2,FUM
0,-0.206692,-0.000000,-24.820305,0.018618,-0.5,0.000000,2.235762,-2.196026,0.500000,0.000000,0.0,0.738815,0.055106,0.000000,0.0,0.0,0.738212
1,-0.346112,-0.000000,-19.097431,0.062354,-1.0,0.000000,3.440514,-3.307436,1.000000,0.000000,0.0,1.125274,0.184554,0.000000,0.0,0.0,1.123254
2,-0.392545,-0.000000,-17.191454,0.106079,-1.5,0.000000,4.645712,-4.419314,1.500000,0.000000,0.0,1.511885,0.313970,0.000000,0.0,0.0,1.508448
3,-0.415374,-0.000000,-16.254365,0.149664,-2.0,0.000000,5.856639,-5.537219,1.911361,0.088639,0.0,1.898786,0.462683,0.000000,0.0,0.0,1.893937
4,-0.427383,-0.000000,-15.761410,0.192489,-2.5,0.000000,7.098777,-6.687958,2.385998,0.114002,0.0,2.446234,0.743765,0.000000,0.0,0.0,2.439997
5,-0.435157,-0.000000,-15.442336,0.235188,-3.0,0.000000,8.346083,-7.844134,2.715818,0.284182,0.0,2.953818,1.031122,0.000000,0.0,0.0,2.946198
6,-0.436441,-0.000000,-15.389601,0.275196,-3.5,0.000000,9.703845,-9.116509,3.022183,0.477817,0.0,3.452768,0.586598,0.685499,0.0,0.0,3.443851
7,-0.425920,-0.584139,-14.653195,0.306928,-4.0,0.420944,10.559437,-9.904378,3.467087,0.532913,0.0,3.622805,0.000000,1.418779,0.0,0.0,3.612860
8,-0.417298,-1.197238,-13.780907,0.338304,-4.5,0.970603,11.172201,-10.450177,3.912610,0.587390,0.0,3.668713,0.000000,1.563816,0.0,0.0,3.657752
9,-0.410401,-1.687717,-13.083076,0.369681,-5.0,1.520261,11.784966,-10.995977,4.358132,0.641868,0.0,3.714620,0.000000,1.708852,0.0,0.0,3.702643


/home/samiralvdb/Documents/3_Projects/2_PAMpy/2_PAMpy_Gitlab/Examples/../Package/PAMpy/PAMValidator.py:105: FutureWarning:

This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.

/home/samiralvdb/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning:

Solver status is 'infeasible'.



Running simulations to validate the model...



,Y_biomass,Y_acetate,Y_co2,BIOMASS_Ec_iML1515_core_75p37M,EX_glc__D_e,EX_ac_e,EX_co2_e,EX_o2_e,PGI,G6PDH2r,EDA,CS,ICL,PPC,ME1,ME2,FUM
0,-0.206692,-0.000000,-24.820305,0.018618,-0.5,0.000000,2.235762,-2.196026,0.500000,0.000000,0.0,0.738815,0.055106,0.000000,0.0,0.0,0.738212
1,-0.346112,-0.000000,-19.097431,0.062354,-1.0,0.000000,3.440514,-3.307436,1.000000,0.000000,0.0,1.125274,0.184554,0.000000,0.0,0.0,1.123254
2,-0.392545,-0.000000,-17.191454,0.106079,-1.5,0.000000,4.645712,-4.419314,1.500000,0.000000,0.0,1.511885,0.313970,0.000000,0.0,0.0,1.508448
3,-0.415374,-0.000000,-16.254365,0.149664,-2.0,0.000000,5.856639,-5.537219,1.911361,0.088639,0.0,1.898786,0.462683,0.000000,0.0,0.0,1.893937
4,-0.427383,-0.000000,-15.761410,0.192489,-2.5,0.000000,7.098777,-6.687958,2.385998,0.114002,0.0,2.446234,0.743765,0.000000,0.0,0.0,2.439997
5,-0.435157,-0.000000,-15.442336,0.235188,-3.0,0.000000,8.346083,-7.844134,2.715818,0.284182,0.0,2.953818,1.031122,0.000000,0.0,0.0,2.946198
6,-0.436441,-0.000000,-15.389601,0.275196,-3.5,0.000000,9.703845,-9.116509,3.022183,0.477817,0.0,3.452768,0.586598,0.685499,0.0,0.0,3.443851
7,-0.425920,-0.584139,-14.653195,0.306928,-4.0,0.420944,10.559437,-9.904378,3.467087,0.532913,0.0,3.622805,0.000000,1.418779,0.0,0.0,3.612860
8,-0.417298,-1.197238,-13.780907,0.338304,-4.5,0.970603,11.172201,-10.450177,3.912610,0.587390,0.0,3.668713,0.000000,1.563816,0.0,0.0,3.657752
9,-0.410401,-1.687717,-13.083076,0.369681,-5.0,1.520261,11.784966,-10.995977,4.358132,0.641868,0.0,3.714620,0.000000,1.708852,0.0,0.0,3.702643


Plotting the results against reference values. 



## 3. Changing carbon source

The model cannot only do simulations for glucose limiting conditions. Also other carbon sources can be used. However, the translational and unused protein sector are calibrated for glucose limited conditions. We recommend recalibrating these sectors for the desired carbon source for more accurate simulations. *see Tranlational_sector_config.ipynb in the scripts folder*

Here we will use glycerol limitation as an example.

In [6]:
#create the model dependend on glycerol uptake
#load the genome-scale information
model = read_sbml_model(os.path.join(MODEL_DIR, 'iML1515.xml'))

#load the unused enzyme information
unused_protein_info = pd.read_excel(pam_info_file, sheet_name='ExcessEnzymes')

ups_0 = unused_protein_info[unused_protein_info.Parameter == 'ups_0'].loc[2,'Value']
smax = unused_protein_info[unused_protein_info.Parameter == 's_max_uptake'].loc[1,'Value']
# id_list = ['EX_glyc_e']
id_list =[unused_protein_info[translational_info.Parameter == 'id_list'].loc[0,'Value']]


unused_protein_sector_glyc = PAMpy.UnusedEnzymeSector(id_list=['EX_glyc_e'],
                                            ups_mu=[ups_0/smax],
                                            ups_0=[ups_0],
                                            mol_mass=[unused_protein_info[unused_protein_info.Parameter == 'mol_mass'].loc[3,'Value']])

#load the PAM with the genome-scale information and the information about the enzyme sectors
pamodel_glyc = PAMpy.PAModel(id_or_model=model, 
                  p_tot=TOTAL_PROTEIN_CONCENTRATION,
                   active_sector=active_enzyme_sector, 
                  translational_sector=translation_enzyme_sector,
                  unused_sector=unused_protein_sector_glyc,
                        sensitivity =False
                 )

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-07
Read LP format model from file /tmp/tmpy6lq582p.lp
Reading time = 0.01 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Setting up the proteome allocation model iML1515

Add total condition-dependent protein constraint
	Total protein concentration: 0.258 g/gDW

Add active protein sector



/home/samiralvdb/Documents/3_Projects/2_PAMpy/2_PAMpy_Gitlab/Examples/../Package/PAMpy/PAModel.py:211: UserWarning: Molar mass for E332 is invalid: 0.0
  warnings.warn(f'Molar mass for {enz.id} is invalid: {molmass}')


Add the following protein sector: TranslationalProteinSector

Add the following protein sector: UnusedEnzymeSector

Done with setting up the proteome allocation model iML1515



In [7]:
validator = PAMpy.PAMValidator(pamodel_glyc, valid_data_file_glyc)
#model flux rates of biomass formation, acetate, CO2 and O2 vs glucose uptake rate for a range of growth rates
validator.validate_range(c_uptake_rxn = 'EX_glyc_e')

Running simulations to validate the model...



/home/samiralvdb/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/samiralvdb/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


BIOMASS_Ec_iML1515_core_75p37M     0.025561
Y_biomass                          5.550749
EX_glyc_e                         -1.000000
EX_ac_e                            0.000000
Y_acetate                         -0.000000
EX_co2_e                           1.950852
Y_co2                            -10.828686
EX_o2_e                           -2.396305
PGI                                0.000000
G6PDH2r                            0.000000
EDA                                0.000000
CS                                 0.641453
ICL                                0.075665
PPC                                0.000000
ME1                                0.000000
ME2                                0.000000
dtype: float64
BIOMASS_Ec_iML1515_core_75p37M    0.025561
EX_glyc_e                        -1.000000
EX_ac_e                           0.000000
EX_co2_e                          1.950852
EX_o2_e                          -2.396305
PGI                               0.000000
G6PDH2r                

BIOMASS_Ec_iML1515_core_75p37M     0.385490
Y_biomass                          5.550749
EX_glyc_e                         -8.000000
EX_ac_e                            0.000000
Y_acetate                         -0.000000
EX_co2_e                           8.176550
Y_co2                             -5.673246
EX_o2_e                          -11.353820
PGI                                0.000000
G6PDH2r                            0.000000
EDA                                0.000000
CS                                 2.714207
ICL                                1.241931
PPC                                0.000000
ME1                                0.000000
ME2                                0.000000
dtype: float64
BIOMASS_Ec_iML1515_core_75p37M     0.385490
EX_glyc_e                         -8.000000
EX_ac_e                            0.000000
EX_co2_e                           8.176550
EX_o2_e                          -11.353820
PGI                                0.000000
G6PDH2r          

,BIOMASS_Ec_iML1515_core_75p37M,EX_glyc_e,EX_ac_e,EX_co2_e,EX_o2_e,PGI,G6PDH2r,EDA,CS,ICL,PPC,ME1,ME2
0,0.025561,-1.0,0.0,1.950852,-2.396305,0.0,0.0,0.0,0.641453,0.075665,0.0,0.0,0.0
1,0.051522,-1.5,0.0,2.385257,-3.025308,0.0,0.0,0.0,0.777286,0.152517,0.0,0.0,0.0
2,0.077495,-2.0,0.0,2.819199,-3.653823,0.0,0.0,0.0,0.912961,0.229402,0.0,0.0,0.0
3,0.103450,-2.5,0.0,3.253612,-4.282825,0.0,0.0,0.0,1.048761,0.306190,0.0,0.0,0.0
4,0.129144,-3.0,0.0,3.698953,-4.923328,0.0,0.0,0.0,1.188321,0.382237,0.0,0.0,0.0
5,0.154837,-3.5,0.0,4.144292,-5.563832,0.0,0.0,0.0,1.327882,0.458284,0.0,0.0,0.0
6,0.180531,-4.0,0.0,4.589632,-6.204336,0.0,0.0,0.0,1.467443,0.534331,0.0,0.0,0.0
7,0.206224,-4.5,0.0,5.034972,-6.844839,0.0,0.0,0.0,1.607004,0.610378,0.0,0.0,0.0
8,0.231918,-5.0,0.0,5.480312,-7.485343,0.0,0.0,0.0,1.746565,0.686426,0.0,0.0,0.0
9,0.257612,-5.5,0.0,5.925652,-8.125846,0.0,0.0,0.0,1.886126,0.762473,0.0,0.0,0.0


Plotting the results against reference values. 



## 4. Adding or removing enzymes, reactions or sections
After building the model, we can modify it. For example by adding or removing components or by changing the kcat values. More modifications are possible, eg. changing the protein-reaction relations, please have a look in the source code on how they work.

### 4.1 Enzymes

In [8]:
#removing the enzymes related to the fumarase reaction from the TCA cycle (fum_c + h2o_c ⇌ mal__L_c)
rxn_id = 'FUM'

fum_enzymes = pamodel.get_enzymes_with_reaction_id(rxn_id)
fum_enzymes_copy = fum_enzymes.copy()
fum_enzyme_ids = [enz.id for enz in fum_enzymes]
print(f'These are the enzymes associated with {rxn_id}: {fum_enzyme_ids}\n')
#remove the enzyme from the model
pamodel.remove_enzymes(fum_enzymes)

#check if the protein is removed from the total protein constraint
tpc = pamodel.constraints[pamodel.TOTAL_PROTEIN_CONSTRAINT_ID]
for enz_id in fum_enzyme_ids:
    if pamodel.enzymes.has_id(enz_id):
        print(f'Enzyme {enz_id} is still in the model!\n')
    else:
        print(f'Enzyme {enz_id} is removed from the model!\n')

#did the model result change?
pamodel.test()

These are the enzymes associated with FUM: ['4.2.1.2']

Enzyme 4.2.1.2 is removed from the model!

Protein allocation model test case was successful.

Optimal objective function value:  0.6523617852897537 

Predicted glucose uptake rate:  -10.0


In [9]:
#adding the enzymes back to the model
pamodel.add_enzymes(fum_enzymes_copy)

#check if the protein is removed from the total protein constraint
tpc = pamodel.constraints[pamodel.TOTAL_PROTEIN_CONSTRAINT_ID]

for enz_id in fum_enzyme_ids:
    if pamodel.enzymes.has_id(enz_id):
        print(f'Enzyme {enz_id} is in the model!\n')
    else:
        print(f'Enzyme {enz_id} is not in the model!\n')

#did the model result change?
pamodel.test()

Enzyme 4.2.1.2 is in the model!

Protein allocation model test case was successful.

Optimal objective function value:  0.6828394923127412 

Predicted glucose uptake rate:  -10.0


In [11]:
#we can also change the enzyme bounds

enz = pamodel.enzymes[1]
min = -enz._constraints[enz.id+'_min'].ub
max = enz._constraints[enz.id+'_max'].ub

print(enz._constraints[enz.id+'_max'])
print(enz._constraints[enz.id+'_min'])

change_enzyme_bounds(pamodel, enz.id, 10,100)

print(enz._constraints[enz.id+'_max'])
print(enz._constraints[enz.id+'_min'])

#change back
change_enzyme_bounds(pamodel, enz.id, min,max)

1.3.1.76_max: -1000000.0 <= 1.0*1.3.1.76 + 1.0*1.3.1.76_reverse_7c2ff <= 100
1.3.1.76_min: -1000000.0 <= -1.0*1.3.1.76 - 1.0*1.3.1.76_reverse_7c2ff <= -10
1.3.1.76_max: -1000000.0 <= 1.0*1.3.1.76 + 1.0*1.3.1.76_reverse_7c2ff <= 100
1.3.1.76_min: -1000000.0 <= -1.0*1.3.1.76 - 1.0*1.3.1.76_reverse_7c2ff <= -10


### 4.2 Reactions

In [8]:
#removing a reaction and the associated catalytic event fom the model
rxn_id = 'FUM'
rxn = pamodel.reactions.get_by_id(rxn_id)
fum_enzymes = pamodel.get_enzymes_with_reaction_id(rxn_id)
catalytic_event = [pamodel.catalytic_events.get_by_id('CE_'+rxn_id)].copy()

pamodel.remove_reactions(pamodel.reactions.get_by_id(rxn_id))

#check if the associated catalytic event is still in the variables of the model
try:
    var = pamodel.variables[catalytic_event[0].forward_variable]
    print(f'Variable {catalytic_event[0].id} is still in the model!\n')
except:
    print(f'Variable {catalytic_event[0].id} is removed from the model!\n')

#did the model result change?
pamodel.test()

['CE_DTARTD']
Variable CE_FUM is removed from the model!

Protein allocation model test case was successful.

Optimal objective function value:  0.6523610560719434 

Predicted glucose uptake rate:  -10.0


In [9]:
#adding the reaction back to the model
pamodel.add_reactions([rxn])

In [10]:
#adding the associated catalytic events to the model
pamodel.add_catalytic_events([catalytic_event[0]])

#check if enzymes are also in the model and associated with the total protein constraint:
tpc = pamodel.constraints[pamodel.TOTAL_PROTEIN_CONSTRAINT_ID]
for enz in fum_enzymes:
    try:
        pamodel.enzymes.get_by_id(enz.id)
        print(f'Enzyme {enz.id} is in the model\n')
    except:
        print(f'Enzyme {enz.id} is NOT in the model\n')
        
#did the results revert?
pamodel.test()

Enzyme 4.2.1.2 is in the model
Variable CE_DTARTD is NOT in the total protein constraint!
Variable CE_FUM is NOT in the total protein constraint!
Protein allocation model test case was successful.

Optimal objective function value:  0.6851081904945704 

Predicted glucose uptake rate:  -10.0


### 4.3 Sectors

In [16]:
#removing a sector from the model
ups = pamodel.sectors.get_by_id('UnusedEnzymeSector')
pamodel.remove_sectors([ups])

#check the results: do they change upon removal of the enzyme sector?
pamodel.test()

#check if the protein sector is still related to the total protein constraint in the model
tpc = pamodel.constraints[pamodel.TOTAL_PROTEIN_CONSTRAINT_ID]
lin_rxn = pamodel.reactions.get_by_id(ups.id_list[0])
if tpc.get_linear_coefficients([lin_rxn.forward_variable])[lin_rxn.forward_variable]>0:
    print(f'The {ups.id} is still associated with the total protein constraint')
else:
    print(f'The {ups.id} is removed from the total protein constraint')

Removing the following protein sector: UnusedEnzymeSector

Protein allocation model test case was successful.

Optimal objective function value:  0.6753974498883657 

Predicted glucose uptake rate:  -10.0
The UnusedEnzymeSector is removed from the total protein constraint


In [17]:
#adding the sector to the model again
pamodel.add_sectors([ups])

#check the results: do they change upon addition of the enzyme sector?
pamodel.test()

#check if the protein sector is related to the total protein constraint in the model
tpc = pamodel.constraints[pamodel.TOTAL_PROTEIN_CONSTRAINT_ID]
lin_rxn = pamodel.reactions.get_by_id(ups.id_list[0])
if tpc.get_linear_coefficients([lin_rxn.forward_variable])[lin_rxn.forward_variable]>0:
    print(f'The {ups.id} is associated with the total protein constraint')
else:
    print(f'The {ups.id} is not associated with the total protein constraint')


Add the following protein sector: UnusedEnzymeSector

Protein allocation model test case was successful.

Optimal objective function value:  0.6828400192152799 

Predicted glucose uptake rate:  -10.0
The UnusedEnzymeSector is associated with the total protein constraint


## 5. Change kcat values

In [8]:
#get the enzymes related to the reaction of interest
ce = pamodel.catalytic_events.get_by_id('CE_FUM')
#get the enzyme which kcat we want to change
enzyme = ce.enzymes[0]

print(enzyme._constraints[f'EC_{enzyme.id}_f']) #for validation

#change kcat and print the output to see if our change worked
ce.change_kcat_values({enzyme.id:{'f':0.1}})
print(enzyme._constraints[f'EC_{enzyme.id}_f'], '\n')

#did result change?
pamodel.test()

EC_4.2.1.2_f: 0 <= -1.0*4.2.1.2 + 0.241650959354309*DTARTD + 4.62962962962963*FUM <= 0
EC_4.2.1.2_f: 0 <= -1.0*4.2.1.2 + 0.241650959354309*DTARTD + 2777.77777777778*FUM <= 0 

Protein allocation model test case was successful.

Optimal objective function value:  0.652361783183415 

Predicted glucose uptake rate:  -10.0


In [9]:
#or even easier, change it right from the PAModel
#first create an imput with enzyme and reaction ids
print(enzyme._constraints[f'EC_{enzyme.id}_f']) #for validation
to_change = {'FUM':{'f': 0.5}}

pamodel.change_kcat_value(enzyme.id, to_change)
#did the change work?
print(enzyme._constraints[f'EC_{enzyme.id}_f'], '\n')

#did result change?
pamodel.test()

EC_4.2.1.2_f: 0 <= -1.0*4.2.1.2 + 0.241650959354309*DTARTD + 2777.77777777778*FUM <= 0
EC_4.2.1.2_f: 0 <= -1.0*4.2.1.2 + 0.241650959354309*DTARTD + 555.555555555556*FUM <= 0 

Protein allocation model test case was successful.

Optimal objective function value:  0.6523617831834134 

Predicted glucose uptake rate:  -10.0
